# Introduction to Python Flask
# Step-by-Step Guide: Building a Flask Web App with SQLite3

## Overview
Flask is a lightweight web framework for Python that makes it easy to create web applications. It is widely used for developing APIs and small to medium-scale web applications.

In this guide, we will create a simple web application using Flask and SQLite3. The application will include:
- A user authentication system (login & register)
- A database connection to store users and items (e.g., computer games)
- Routes to add, view, and delete items using HTML templates

## Learning Objectives
- Understand the basics of Flask
- Set up and run a Flask application
- Create and manage routes
- Connect Flask to an SQLite database
- Handle CRUD operations using Flask and SQLite

---

## Understanding Flask Routes
Flask uses **routes** to define different pages or API endpoints in a web application. A route is defined using the `@app.route()` decorator. Routes can handle different HTTP methods like `GET` (fetch data) and `POST` (send data).

### Example Route:

In [ ]:
@app.route('/')  # This defines the homepage
def home():
    return "Welcome to the Flask App!"

### Routes Handling GET/POST Requests
- `GET` method: Used to retrieve data from the server.
- `POST` method: Used to send data to the server (e.g., submitting a form).

Example of handling both `GET` and `POST` requests:

In [ ]:
@app.route('/submit', methods=['GET', 'POST'])
def submit():
    if request.method == 'POST':
        data = request.form['name']  # Retrieve data from form input
        return f"Hello, {data}!"
    return render_template('submit.html')  # Show form page

## Step-by-Step Guide: Building a Flask Web App with SQLite3
***

### 1. Installing Flask
To get started, install Flask using pip:
```python
pip install flask

### Step 2: Setting Up the Database

Create a `db_connector.py` file:

In [ ]:
import sqlite3  # Import SQLite library

class Database:
    def __init__(self):
        # Define the database name
        self.DBname = 'SQLtask.db'

    # Create a connection to the database
    def connect(self):
        conn = None
        try:
            conn = sqlite3.connect(self.DBname)  # Connect to the database
        except Exception as e:
            print(e)  # Print any connection error
        return conn

    # Execute a SELECT query
    def queryDB(self, command, params=[]):
        conn = self.connect()
        cur = conn.cursor()
        cur.execute(command, params)  # Execute query
        result = cur.fetchall()  # Fetch results
        self.disconnect(conn)  # Close connection
        return result

    # Execute an INSERT, UPDATE, or DELETE query
    def updateDB(self, command, params=[]):
        conn = self.connect()
        cur = conn.cursor()
        cur.execute(command, params)  # Execute update
        conn.commit()  # Commit changes
        result = cur.fetchall()
        self.disconnect(conn)  # Close connection
        return result

    # Close the database connection
    def disconnect(self, conn):
        conn.close()


---

### Step 3: Setting Up Flask

Create `app.py` with the following structure:

In [ ]:
from flask import Flask, request, render_template, redirect, url_for, session
from flask_bcrypt import Bcrypt
from flask_login import LoginManager, login_user, login_required, logout_user
from db_connector import Database

# Initialize Flask app
app = Flask(__name__)
app.secret_key = 'supersecretkey'  # Secret key for session management
bcrypt = Bcrypt(app)
db = Database()

# Initialize login manager for authentication
login_manager = LoginManager()
login_manager.init_app(app)  # Attach login manager to the app

if __name__ == '__main__':
    app.run(debug=True)  # Run Flask in debug mode


### Step 4: User Authentication

### Example: Using GET and POST Methods with Forms

In [ ]:
@app.route('/add_game', methods=['GET', 'POST'])
def add_game():
    if request.method == 'POST':
        title = request.form['title']
        genre = request.form['genre']
        platform = request.form['platform']
        db.updateDB("INSERT INTO games (title, genre, platform) VALUES (?, ?, ?)", (title, genre, platform))
        return redirect(url_for('dashboard'))  # Redirect to updated dashboard
    return render_template('add_game.html')  # Show form page

#### Register a New User

In [ ]:
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']  # Retrieve username from form
        password = bcrypt.generate_password_hash(request.form['password']).decode('utf-8')  # Hash password
        
        try:
            # Insert new user into the database
            db.updateDB("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
            return redirect(url_for('login'))  # Redirect to login page
        except:
            return render_template('register.html', message="Username already exists")
    
    return render_template('register.html')  # Render registration form

### Login User

In [ ]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']  # Retrieve username
        password = request.form['password']  # Retrieve password
        
        user = db.queryDB("SELECT * FROM users WHERE username = ?", (username,))  # Fetch user from DB
        
        # Check if user exists and password matches
        if user and bcrypt.check_password_hash(user[0][2], password):
            session['user'] = username  # Store user session
            return redirect(url_for('dashboard'))  # Redirect to dashboard
        else:
            return render_template('login.html', message="Invalid credentials")
    
    return render_template('login.html')  # Render login form

### Logout User

In [ ]:
@app.route('/logout')
@login_required  # Ensure only logged-in users can log out

def logout():
    logout_user()  # Logout user
    return redirect(url_for('login'))  # Redirect to login page

### Step 5: Handling Game Data

### Dashboard - View All Games

In [ ]:
@app.route('/dashboard')
@login_required  # Only authenticated users can access

def dashboard():
    games = db.queryDB("SELECT * FROM games")  # Fetch all games from DB
    return render_template('dashboard.html', games=games)  # Render dashboard page

### Add a New Game

In [ ]:
@app.route('/add_game', methods=['GET', 'POST'])
@login_required  # Only authenticated users can add games

def add_game():
    if request.method == 'POST':
        title = request.form['title']  # Retrieve game title
        genre = request.form['genre']  # Retrieve game genre
        platform = request.form['platform']  # Retrieve game platform
        
        # Insert game into database
        db.updateDB("INSERT INTO games (title, genre, platform) VALUES (?, ?, ?)", (title, genre, platform))
        return redirect(url_for('dashboard'))  # Redirect to dashboard
    
    return render_template('add_game.html')  # Render add game form

### Delete a Game

In [ ]:
@app.route('/delete_game/<int:game_id>')
@login_required  # Only authenticated users can delete games

def delete_game(game_id):
    # Delete game by ID
    db.updateDB("DELETE FROM games WHERE id = ?", (game_id,))
    return redirect(url_for('dashboard'))  # Redirect to dashboard

***
### Step 6: Running the Application

Run `app.py`

The following code is executed:

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)  # Run Flask in debug mode

Access `http://127.0.0.1:5000/` to use the application.

## Understanding Jinja2 Templating
Jinja2 is Flask’s template engine that allows embedding Python-like code in HTML.

### Key Jinja2 Features:
1. **Variables:** `{{ variable_name }}` → Displays data from Python in the template.
2. **Loops:** `{% for item in list %} ... {% endfor %}` → Iterates through lists.
3. **Conditionals:** `{% if condition %} ... {% else %} ... {% endif %}` → Adds logic.

### Example Usage:


In [ ]:
<ul>
    {% for game in games %}
    <li>{{ game.title }} - {{ game.genre }} ({{ game.platform }})</li>
    {% endfor %}
</ul>

This will loop through `games` passed from the Flask app and display each game’s details.

***
### Step 7: Creating HTML Templates

## Understanding the Base Template (`base.html`)
Flask uses **template inheritance** to avoid code repetition. A **base template** defines common page structures like headers, footers, and navigation bars.

### Example `base.html` (Used by all other pages):

### `templates/base.html`
html

In [3]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Flask App{% endblock %}</title>
</head>
<body>
    <!-- Navigation Bar with links to Dashboard and Logout -->
    <nav>
        <a href="{{ url_for('dashboard') }}">Dashboard</a> |
        <a href="{{ url_for('logout') }}">Logout</a>
    </nav>
    
    <!-- This block will be replaced by content from child templates -->
    {% block content %}{% endblock %}
</body>
</html>

SyntaxError: invalid syntax (1702565060.py, line 1)

### `templates/login.html`
html

In [ ]:
{% extends 'base.html' %}
{% block title %}Login{% endblock %}
{% block content %}
<h2>Login</h2>
<form method="POST">
    <!-- Input field for username -->
    <input type="text" name="username" placeholder="Username" required>
    
    <!-- Input field for password -->
    <input type="password" name="password" placeholder="Password" required>
    
    <!-- Submit button to log in -->
    <button type="submit">Login</button>
</form>

<!-- Display error message if login fails -->
{% if message %}<p>{{ message }}</p>{% endif %}
{% endblock %}

### `templates/register.html`
html

In [ ]:
{% extends 'base.html' %}
{% block title %}Register{% endblock %}
{% block content %}
<h2>Register</h2>
<form method="POST">
    <!-- Input field for username -->
    <input type="text" name="username" placeholder="Username" required>
    
    <!-- Input field for password -->
    <input type="password" name="password" placeholder="Password" required>
    
    <!-- Submit button to register -->
    <button type="submit">Register</button>
</form>

<!-- Display message if registration fails -->
{% if message %}<p>{{ message }}</p>{% endif %}
{% endblock %}

### `templates/dashboard.html`
html

In [ ]:
{% extends 'base.html' %}
{% block title %}Dashboard{% endblock %}
{% block content %}
<h2>Game List</h2>
<a href="{{ url_for('add_game') }}">Add Game</a>
<ul>
    {% for game in games %}
    <li>
        <!-- Display game title, genre, and platform -->
        {{ game[1] }} - {{ game[2] }} ({{ game[3] }}) 
        
        <!-- Link to delete the game -->
        <a href="{{ url_for('delete_game', game_id=game[0]) }}">Delete</a>
    </li>
    {% endfor %}
</ul>
{% endblock %}

### `templates/add_game.html`
html

In [ ]:
{% extends 'base.html' %}
{% block title %}Add Game{% endblock %}
{% block content %}
<h2>Add New Game</h2>
<form method="POST">
    <!-- Input field for game title -->
    <input type="text" name="title" placeholder="Game Title" required>
    
    <!-- Input field for game genre -->
    <input type="text" name="genre" placeholder="Genre" required>
    
    <!-- Input field for game platform -->
    <input type="text" name="platform" placeholder="Platform" required>
    
    <!-- Submit button to add the game -->
    <button type="submit">Add Game</button>
</form>
{% endblock %}

***

## Summary
In this guide, we:
- Set up a Flask project
- Created a database with SQLite
- Implemented user authentication (register, login, logout)
- Created routes using `render_template` and `redirect`
- Managed game data using HTML templates
- Added an `init_db.py` script to initialize the database
***
### Final Task
Enhance the project by:
1. Adding an edit/update route for games
2. Creating a frontend to interact with the API
3. Implementing user roles (admin & normal user)